# Introduction

The *pyveg* package contains some useful functions for interacting with the Python API of Google Earth Engine to download images, and also code to process these images and prepare them for analysis.

In particular, we want to look at the "connectedness" of patterned vegetation.  To do this, we download NDVI (Normalised Difference Vegetation Index) images from GEE, and use some image processing techniques to convert these into binary black-and-white images, that we then divide into 50x50 pixel sub-images, and do some network analysis on them.

Before we can use GEE, we need to authenticate (assuming we have an account).

In [ ]:
import ee
ee.Authenticate()

## Pipelines, Sequences, and Modules

In pyveg, we have the concept of a "Pipeline" for downloading and processing data from GEE.

A Pipeline is composed of one or more Sequences, which are in turn composed of Modules.

A Module is an class designed for one specific task (e.g. "download vegetation data from GEE", or "calculate network centrality of binary images"), and they are generally grouped into Sequences such that one Module will work on the output of the previous one.  
So our standard Pipeline has:
* A vegetation Sequence consisting of VegetationDownloader, VegetationImageProcessor, NetworkCentralityCalculator, and NDVICalculator.   
* A weather Sequence consisting of WeatherDownloader, WeatherImageToJSON
* A combiner Sequence consisting of a single combiner Module, that takes the outputs of the other two Sequences and produces a final output file.

### Running the full pipeline from the command-line

For the second part of this notebook will will demonstrate running individual Modules and Sequences, but the majority of users will probably just want to run the full Pipeline for their selected location/collection/date range, so we will cover that first.

We have a couple of "entrypoints" (i.e. command-line commands) linked to functions in some pyveg scripts to help do this.  
* To configure and run a downloading-and-processing pipeline we run the command `pyveg_run_pipeline --config_file <some-config-file>`
* To generate the config file in the above command we have the command `pyveg_generate_config`.

Both these can accept multiple command-line arguments, and these can be seen with the `--help` argument:

In [ ]:
!pyveg_generate_config --help

For `pyveg_generate_config` any parameters it needs that are not provided as command-line arguments will be requested from the user, and the various allowed options will be provided, along with (in most cases) default values that will be used if the user just presses "enter".
However, although just running `pyveg_generate_config` with no arguments and then responding to the prompts is probably the easiest way to run it on the command line, this doesn't seem to work so well with Jupyter, so let's just provide all the arguments it needs:

In [ ]:
!pyveg_generate_config --configs_dir ../../pyveg/configs --collection_name Sentinel2 --output_dir ./ --test_mode --latitude 11.58 --longitude 27.94 --country Sudan --start_date 2019-01-01 --end_date 2019-04-01 --time_per_point 1m --run_mode local --n_threads 2

We can see from the output that a new config file has been written, and the command we should use to run with it.

In [ ]:
!pyveg_run_pipeline --config_file ../../pyveg/configs/testconfig_Sentinel2_11.58N_27.94E_Sudan_2019-01-01_2019-04-01_1m_local.py




So we just:
* Downloaded some Sentinel2 images from GEE
* Converted these raw tif images into RGB png, greyscale NDVI png, and black-and-white binarized png images.
* Split the above pngs into 50x50 sub-images
* Calculated the Network Centrality and total NDVI of each sub-image
* Downloaded some ERA5 weather data from GEE
* Read off the values of precipitation and temperature from these tifs
* Combined the vegetation and weather data into one output file

The final output file is called "results_summary.json" and contains some metadata describing the configuration, and time-series data for the vegetation and weather.

# (Optional) Running the pieces individually

Though the above method is the easiest way to get up-and-running, some users may be interested in running the components of pyveg individually.

In [ ]:
from pyveg.src.download_modules import VegetationDownloader

In [ ]:
# instantiate this Module:
vd = VegetationDownloader("Sentinel2_download")

A lot of the parameters we need to configure this Module are in the `configs/collections.py` file - there is a large dictionary containing values for e.g. Sentinel 2.

In [ ]:
from pyveg.configs.collections import data_collections
s2_config = data_collections["Sentinel2"]
print(s2_config)

we also need to specify the coordinates we want to look at (in ***(long,lat)*** format) - let's look at one of our locations in the Sahel:

In [ ]:
coords = [28.37,11.12]

And we need to choose a date range.  If we are looking at vegetation data as in this case, we will take the median of all images available within this date range (after filtering out cloudy ones).

For the sake of this tutorial, let's just look at a short date range - in fact just a single month:

In [ ]:
date_range = ["2018-06-01","2018-07-01"]

We also need to set an output location to store the files.  We can just use a temporary directory.   The downloaded files will go into a subdirectory of this called "RAW", and then into further subdirectories per mid-point of each date sub-range we're looking at.   Here, we are just looking at one month, and the midpoint will be "2018-06-16".

In [ ]:
import os
if os.name == "posix":
    TMPDIR = "/tmp"
else:
    TMPDIR = "%TMP%"
    
output_veg_location = os.path.join(TMPDIR,"gee_veg_download_example")
output_location_type = "local" # other alternative currently possible is `azure` for MS Azure cloud, if setup

Now we're ready to configure the module:

In [ ]:
# we could go through all the key,value pairs in the s2_config dict setting them all
# individually, but lets do them all at once
vd.set_parameters(s2_config)
vd.coords = coords
vd.date_range = date_range
vd.output_location = output_veg_location
vd.output_location_type = output_location_type
vd.configure()
print(vd)

The Module is all configured and ready-to-go!

In [ ]:
vd.run()

There should now be some files in the output location:

In [ ]:
os.listdir(os.path.join(output_veg_location,"2018-06-16","RAW"))

So we have one .tif file per band.   

The next Module we would normally run in the vegetation Sequence is the VegetationImageProcessor that will take these tif files and produce png images from them.  This includes histogram equalization, adaptive thresholding and median filtering on an input image, to give us binary NDVI images.  It then divides these into 50x50 sub-images.

In [ ]:
from pyveg.src.processor_modules import VegetationImageProcessor
vip = VegetationImageProcessor("Sentinel2_img_processor")
vip.set_parameters(s2_config)
vip.coords = coords


The only other things we need to set are the `input_location` (which will be the `output_location` from the downloader), and the `output_location` (which we will put as the same as the downloader's one - the results of this will go into different subdirectories of the date-named subdirectories).

In [ ]:
vip.input_location = vd.output_location
vip.output_location = vd.output_location
vip.configure()
print(vip)

In [ ]:
vip.run()

This should have created two new subdirectories: "PROCESSED" contains the full-size RGB, greyscale, and black-and-white images (the first of these using the RGB bands, and the latter two based on the NDVI band).  "SPLIT" contains the 50x50 sub-images.

In [ ]:
os.listdir(os.path.join(output_veg_location,"2018-06-16","PROCESSED"))

## Calculating network centrality

The next step in the standard vegetation sequence is the calculation of "offset50", which is related to the "connectedness" of the vegetation in the black-and-white NDVI sub-images.

In [ ]:
from pyveg.src.processor_modules import NetworkCentralityCalculator
ncc = NetworkCentralityCalculator("Sentinel2_ncc")
ncc.set_parameters(s2_config)
ncc.input_location = vip.output_location
ncc.output_location = vip.output_location # same output location again - will create a 'JSON' subdir
ncc.configure()
print(ncc)

One other setting that we might want to change is the number of sub-images per full-size-image for which we do the network centrality calculation.   There are 289 sub-images per full-size-image, and it can be quite time-consuming to process all of them (even though some parallization is implemented - see `n_threads` argument).   We can set this to a smaller number for testing purposes.

In [ ]:
ncc.n_sub_images = 10

In [ ]:
ncc.run()

We should now have a json file in the output directory:

In [ ]:
os.listdir(os.path.join(output_veg_location,"2018-06-16","JSON","NC"))

In [ ]:
import json
j=json.load(open(os.path.join(output_veg_location,"2018-06-16","JSON","NC","network_centralities.json")))


The contents of the json file is a list (one entry per sub-image) of dictionaries, and the dictionary keys includ latitude, longitude of the sub-image, as well as "offset50".

In [ ]:
j[0].keys()

### Running the weather Sequence

Here we ran the vegetation-related Modules one-by-one, but we can also combine Modules into Sequences.  As an example, lets do this for the weather downloader Module, and the Module that reads the downloaded images and produces output json files.

In [ ]:
from pyveg.src.pyveg_pipeline import Sequence
from pyveg.src.download_modules import WeatherDownloader
from pyveg.src.processor_modules import WeatherImageToJSON

In [ ]:
era_config = data_collections["ERA5"]
era_config

The default is to download all the monthly weather data since 1986, but for the sake of speed, lets just look at the same small date range as before

In [ ]:
s=Sequence("era5_sequence")
s.date_range = date_range
s.coords = coords # use the same location as we used above, in the Sahel
s.set_config(era_config)

Now we can add Modules to the Sequence, just using the "+=" operator:

In [ ]:
s += WeatherDownloader()
s += WeatherImageToJSON()
s.configure()
print(s)

We have been given default values for the "output_location", which we might want to override for this example and just use a temporary location

In [ ]:
output_weather_location = os.path.join(TMPDIR, "gee_weather_download_example")
s.output_location = output_weather_location
# need to reconfigure to propagate this to the Modules
s.configure()
print(s)

And we're ready to run!

In [ ]:
s.run()

Let's check we got some output:

In [ ]:
os.listdir(os.path.join(output_weather_location, "2018-06-16","JSON","WEATHER"))

In [ ]:
import json
j=json.load(open(os.path.join(output_weather_location, "2018-06-16","JSON","WEATHER","weather_data.json")))
print(j)